In [1]:
import heapq
import datetime
import numpy as np
from copy import deepcopy

In [2]:
class State:
  def __init__(self, start, pre_move = 'root'):
    self.RHS = start
    self.LHS = [0,0] #[#missionaries, #cannibals]
    self.goal = deepcopy(start)
    self.boad_location = 0 #0 is the RHS or the river, #1 is the LHS of the river
    self.pre_move = pre_move
    self.g = 0
    self.f = self.NumPeopLeft()

  def __eq__(self, other):
    if isinstance(other, State):
      return self.RHS == other.RHS and self.LHS == other.LHS and self.boad_location == other.boad_location
    
  def __hash__(self):
    # h = np.concatenate(self.RHS, self.LHS)
    # h = np.append(h, self.boad_location)
    # h = h.tolist
    # return hash(tuple(h))
    return hash(tuple(self.RHS))
  
  def __lt__(self, other):
    return self.f < other.f

  def ValidMove(self, boad):
    if sum(boad) > 0 and sum(boad) <= 2:

      if self.boad_location == 0: #boad on RHS
        if self.RHS[0] - boad[0] >= 0 and self.RHS[1] - boad[1] >=0: #Prevent over substraction
          if self.RHS[0] - boad[0] == 0: #When one side no M after entering the boad
            if self.LHS[0] + boad[0] >= self.LHS[1] + boad[1]:
              return True
          elif self.LHS[0] + boad[0] == 0: #When one side dont have M after boad reach
            if self.RHS[0] - boad[0] >= self.RHS[1] - boad[1]:
              return True
          elif self.RHS[0] - boad[0] >= self.RHS[1] - boad[1] and self.LHS[0] + boad[0] >= self.LHS[1] + boad[1]:
            return True
          
      if self.boad_location == 1: #boad on LHS
        if self.LHS[0] - boad[0] >= 0 and self.LHS[1] - boad[1] >=0: #Prevent over substraction
          if self.LHS[0] - boad[0] == 0: #When one side no M after entering the boad
            if self.RHS[0] + boad[0] >= self.RHS[1] + boad[1]:
              return True
          elif self.RHS[0] + boad[0] == 0:
            if self.LHS[0] - boad[0] >= self.LHS[1] - boad[1]:
              return True
          elif self.LHS[0] - boad[0] >= self.LHS[1] - boad[1] and self.RHS[0] + boad[0] >= self.RHS[1] + boad[1]:
            return True
           
    return False

  def SavedMove(self, boad):
    saved = deepcopy(self)
    if self.ValidMove(boad):
      if self.boad_location == 0:
        self.RHS[0] -= boad[0]
        self.RHS[1] -= boad[1]
        self.LHS[0] += boad[0]
        self.LHS[1] += boad[1]
        self.pre_move = boad
        self.boad_location = 1
        self.g += 1
      elif self.boad_location == 1:
        self.RHS[0] += boad[0]
        self.RHS[1] += boad[1]
        self.LHS[0] -= boad[0]
        self.LHS[1] -= boad[1]
        self.pre_move = boad
        self.boad_location = 0
        self.g += 1
      return saved
    else:
      print('Invalid move')
      return False 

  def PossibleMoves(self):
    if self.pre_move == 'root':
      return [[2,0],[0,2],[1,1],[1,0],[0,1]]
    if self.pre_move == [2,0]:
      return [[0,2],[1,1],[1,0],[0,1]]
    elif self.pre_move == [0,2]:
      return [[2,0],[1,1],[1,0],[0,1]]
    elif self.pre_move == [1,1]:
      return [[2,0],[0,2],[1,0],[0,1]]
    elif self.pre_move == [1,0]:
      return [[2,0],[0,2],[1,1],[0,1]]
    elif self.pre_move == [0,1]:
      return [[2,0],[0,2],[1,1],[1,0]]

  def Expand(self):
    nodes = []
    for move in self.PossibleMoves():
      if self.ValidMove(move):
        temp = deepcopy(self)
        temp.SavedMove(move)
        nodes.append(temp)
    return nodes
  
  def NumPeopLeft(self):
    return sum(self.RHS)/2

In [3]:
def DFS(state):
    global visited, path, n_expand
    if state in visited:
        return False
    visited[state] = True
    if state.LHS == state.goal:
        print('Solved')
        return True
    else:
        for node in state.Expand():
            n_expand += 1
            found = DFS(node)
            if found:
                path.append(node.pre_move)
                return True
    del visited[state]
    return False

In [4]:
def path_find(history, state):
  path = [state.pre_move]
  temp = state
  while temp.pre_move != 'root':
    temp = history[temp]
    path.append(temp.pre_move)
  return list(reversed(path))[1:]

def ASearch(start):
  global n_expand
  queue = [start]
  Inqueue = {start:True}
  visited = {start:True}
  history = {start:'root'}

  while queue:
    heapq.heapify(queue)
    target = heapq.heappop(queue)
    if target.LHS == target.goal:
      print('Solved')
      return path_find(history, target)
    Inqueue[target] = False
    for node in target.Expand():
      n_expand += 1
      if node not in visited:
        visited[node] = True
        history[node] = target
        if node not in Inqueue:
          heapq.heappush(queue, node)
          Inqueue[node] = True

  return False

In [5]:
def Simulator(moves):
    print('-------------------------------------------------------------------------')
    boad_location = 0
    print(f'Starting on the Right Hand Side, there are 3 Missionaries and 3 Cannibals')
    for move in moves:
        if boad_location == 0:
            print(f'Move {move[0]} Missionaries and {move[1]} Cannibals to the Left Hand Side')
            boad_location = 1
        elif boad_location == 1:
            print(f'Move {move[0]} Missionaries and {move[1]} Cannibals to the Right Hand side')
            boad_location = 0

In [6]:
if __name__ == '__main__':

    visited = {}
    path = []
    n_expand = 0
    
    start_state = [3,3] # First index = #Missionaries, Second index = #Cannibals
    start = State(start_state)

    method = input('Please select an Algorithm \'D\' for DFS, \'A\' for A* Search')
    if method == 'D':
        a = datetime.datetime.now()
        found = DFS(start)
        b = datetime.datetime.now()
        print("Method used: DFS")
        print("Run Time: ", b - a)
        path.reverse()
        print("Path found:", path)
        print('Number of nodes expaned: ', n_expand)
        print('Number of moves:', len(path))
        Simulator(path)
    elif method == 'A':
        a = datetime.datetime.now()
        history = ASearch(start)
        b = datetime.datetime.now()
        print("Method used: A* Search")
        print("Run Time: ", b - a)
        print("Path found:", history)
        print('Number of nodes expaned: ', n_expand)
        print('Number of moves:', len(history))
        Simulator(history)
    else:
        print('Invalid input for Algorithm')
        raise

Solved
Method used: A* Search
Run Time:  0:00:00.001001
Path found: [[0, 2], [0, 1], [0, 2], [0, 1], [2, 0], [1, 1], [2, 0], [0, 1], [0, 2], [0, 1], [0, 2]]
Number of nodes expaned:  15
Number of moves: 11
-------------------------------------------------------------------------
Starting on the Right Hand Side, there are 3 Missionaries and 3 Cannibals
Move 0 Missionaries and 2 Cannibals to the Left Hand Side
Move 0 Missionaries and 1 Cannibals to the Right Hand side
Move 0 Missionaries and 2 Cannibals to the Left Hand Side
Move 0 Missionaries and 1 Cannibals to the Right Hand side
Move 2 Missionaries and 0 Cannibals to the Left Hand Side
Move 1 Missionaries and 1 Cannibals to the Right Hand side
Move 2 Missionaries and 0 Cannibals to the Left Hand Side
Move 0 Missionaries and 1 Cannibals to the Right Hand side
Move 0 Missionaries and 2 Cannibals to the Left Hand Side
Move 0 Missionaries and 1 Cannibals to the Right Hand side
Move 0 Missionaries and 2 Cannibals to the Left Hand Side
